In [1]:
! cd /home/ivan/Uforse/uni_info_generator/ && bash .bashrc
! echo $CACHE_REPO_PATH
! cd /home/ivan/Uforse/uni_info_generator/

Obtaining file:///home/ivan/Uforse/uni_info_generator/university_info_generator
  Preparing metadata (setup.py) ... done
  Attempting uninstall: university-info-generator
    Found existing installation: university-info-generator 0.1.1
    Uninstalling university-info-generator-0.1.1:
      Successfully uninstalled university-info-generator-0.1.1
  Running setup.py develop for university-info-generator
Environment variables set:
PROJECT_HOME=/home/ivan/Uforse/uni_info_generator
PYTHONPATH=/home/ivan/Uforse/uni_info_generator:
API_KEY_PATH=/home/ivan/Uforse/uni_info_generator
CACHE_REPO_PATH=/home/ivan/Uforse/uni_info_generator/cache_repo
Environment set up for uni_info_generator



In [2]:
import sys
print(sys.executable)

/home/ivan/anaconda3/envs/gpu/bin/python


In [3]:
sys.path.append('/home/ivan/Uforse/uni_info_generator')

In [4]:
from university_info_generator import config
from university_info_generator import UniversityInfoGenerator as uni
from university_info_generator import BasicInfoType, SavedDictType, GPTMethodType
from university_info_generator.utility.google_sheet_utility import *
from university_info_generator.utility.save_load_utility import *

In [5]:
attr_df = get_attribute_df()
attr_df.head(25)

,attribute_name,attribute_format,attribute_reference,attribute_prompt,example
0,university_type,string,NaN,Focus on whether the university is public or p...,Public University
1,abbreviation,string,NaN,NaN,UBC
2,graduation_year,int,NaN,"The time for general students, undergraduate, ...",4
3,location,List[str],NaN,"The different locations of the campus, like di...",Vancouver campus\nOkanagan campus
4,graduation_rate,float,NaN,"Keep the format in two digits, like 89 represe...",NaN
5,domestic_student_tuition,string,https://universitystudy.ca/canadian-universities/,Visit https://universitystudy.ca/canadian-univ...,"$6,128 - $7,946'"
6,international_student_tuition,string,https://universitystudy.ca/canadian-universities/,Visit https://universitystudy.ca/canadian-univ...,"$37,038 - $51,448'"
7,description,string,NaN,You could try to extract information from the ...,"The University of British Columbia (UBC), loca..."
8,ranking_qs_news_2024,int,"[""https://www.topuniversities.com/world-univer...",Focus on the latest results from these website...,34
9,ranking_us_news_2023,int,"[""https://www.usnews.com/education/best-global...",Focus on the latest results from these website...,35


In [6]:
store_cache(config.UNIVERSITY_ATTRIBUTE_CACHE_FILE_PATH, get_attribute_dict())

In [7]:
print("CACHE_REPO_PATH:", config.CACHE_REPO_PATH)

CACHE_REPO_PATH: /home/ivan/Uforse/uni_info_generator/cache_repo


In [8]:
import os
import sys
from pprint import pprint

#'/home/ivan/Uforse/uni_info_generator/fetch_logic'
current_path = os.path.abspath('./')

#'/home/ivan/Uforse/uni_info_generator'
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)


In [9]:
uni_gen = uni()
uni_gen.load_from_file(SavedDictType.ATTRIBUTE, config.UNIVERSITY_ATTRIBUTE_CACHE_FILE_PATH)
uni_gen.load_from_file(SavedDictType.GPT_CACHE, "./gpt_cache.jsonl")
uni_gen.load_from_file(SavedDictType.UNIVERSITY_BASIC_INFO, "./university_basic_info.jsonl")
# uni_gen.load_from_file(SavedDictType.UNIVERSITY_INFO, "./university_info.jsonl")


In [10]:
def filter_jsonl_file(input_filepath, output_filepath, keyword):
    """
    Reads a JSONL file and writes out only those lines that do not contain the specified keyword.

    Args:
    - input_filepath (str): Path to the input JSONL file.
    - output_filepath (str): Path to the output JSONL file where results should be saved.
    - keyword (str): Keyword to search for in each JSON object to decide if it should be excluded.
    """
    with open(input_filepath, 'r', encoding='utf-8') as file, open(output_filepath, 'w', encoding='utf-8') as outfile:
        for line in file:
            if keyword not in line:  # Check if the keyword is in the line
                outfile.write(line)  # Write the line to the output file if keyword is not found



In [11]:
# # Example usage
# input_file = 'gpt_cache.jsonl'
# output_file = 'new_gpt_cache.jsonl'
# filter_jsonl_file(input_file, output_file, 'ranking')

In [12]:
target_worksheet = get_worksheet()


In [13]:
target_university = get_worksheet(
    spreadsheet_title= "working_extract_info_output",
    sheetname= "target_university_list",
    sheet_client= get_sheet_client(),
)

In [14]:
def extract_university_names(worksheet):
    # Get all values in the first column (index 1)
    university_names = worksheet.col_values(1)  # This retrieves all data in the first column
    # Optionally, you might want to skip the header or any empty values
    university_names = [name.strip() for name in university_names if name.strip() != '']
    return university_names

In [15]:
lst = extract_university_names(target_university)[1:][:10]
lst.extend(["UW", "UofT", "University Of British Columbia", "University of Calgary"])
pprint(lst)

['Adler Graduate Professional School',
 'Alberta Business School - University of Alberta',
 'Algoma University',
 'Algonquin College',
 'Asper School of Business - University of Manitoba',
 'Assiniboine Community College',
 'Athabasca University',
 'Beedie School of Business - Simon Fraser University',
 "Bishop's University",
 'Bow Valley College',
 'UW',
 'UofT',
 'University Of British Columbia',
 'University of Calgary']


In [24]:
for university in lst:
    uni_gen.get_university_info(university)

In [26]:
uni_gen.save_to_file(SavedDictType.UNIVERSITY_INFO, "./university_info.jsonl")

In [19]:
uni_gen.save_to_file(SavedDictType.UNIVERSITY_BASIC_INFO, "./university_basic_info.jsonl")

In [27]:
uni_gen.save_to_file(SavedDictType.GPT_CACHE, "./gpt_cache.jsonl")

In [28]:
clear_worksheet_content(target_worksheet)

Cleared 14 rows and 22 columns.


In [29]:
write_cache_to_worksheet("university_info.jsonl", target_worksheet)
